In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
# This notebook runs the bronze ingest code

In [0]:
# Databricks notebook source
import sys
import os

# Add repo src directory to path
repo_dir = "/Workspace/Users/arjunn.s@ldsinfotech.com/azure-databricks-retail-lakehouse/src"
sys.path.append(repo_dir)

In [0]:
from bronze.load_data import run_ingestion
import json

def main():
    config_path = "/Workspace/Users/arjunn.s@ldsinfotech.com/azure-databricks-retail-lakehouse/infra/bronze_config.json"

    # Ensure dbutils is available
    try:
        dbutils.fs
    except NameError:
        from pyspark.dbutils import DBUtils
        dbutils = DBUtils(spark)

    config_str = dbutils.fs.head(
        f"file:{config_path}",
        1024 * 1024
    )
    config = json.loads(config_str)

    storage_account = config["storage_account_name"]
    container = config["container_name"]
    
    print(f"✅ Loaded config for {storage_account}/{container}")

    return storage_account, container

if __name__ == "__main__":
    storage_account, container = main()
    account_key = dbutils.secrets.get(scope="initSecret", key="storage-key")
    run_ingestion(storage_account, container, account_key)
    print("✅ Bronze ingest complete")